In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/bbc_data.csv")

In [ ]:
import re

In [ ]:
def chunk_text(text, size=500):
    l = len(text)
    chunked = []
    pos = 0
    while pos < l - 1:
        if pos + size >= l:
            chunked.append(text[pos:l-1])
            pos = l - 1
        else:
            d = re.search('(?<=[+.+!?]) +', text[pos:])
            if d is not None:
                chunked.append(text[pos:pos+ d.end() - 1])
                pos = pos + d.end()

            else:
                chunked.append(text[pos:pos+ size])
                pos = pos + size

    return chunked


In [ ]:
ch= chunk_text(df['data'][0])

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [ ]:
!pip install chromadb

In [ ]:
import chromadb
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/chromadb_text_lab5")

In [ ]:
chroma_client2 = chromadb.PersistentClient(path="/content/drive/MyDrive/chromadb_text_lab5_v2")

chroma_client3 = chromadb.PersistentClient(path="/content/drive/MyDrive/chromadb_text_lab5_v3")

In [ ]:
collection2 = chroma_client.create_collection(name="text")

In [ ]:
collection = chroma_client.get_collection(name="text")
collection.count()

In [ ]:
collection2 = chroma_client2.create_collection(name="text2", metadata={"hnsw:space": "ip"} )

collection3 = chroma_client3.create_collection(name="text3", metadata={"hnsw:space": "cosine"} )

In [ ]:
collection2 = chroma_client2.get_collection(name="text2")

collection3 = chroma_client3.get_collection(name="text3")

In [ ]:
from tqdm import tqdm

In [ ]:
def add_to_collect(df, collection, add_to =False):
    docs, embeds, metas, ids = [], [], [], []

    chunked_text = []

    for i in tqdm(range(len(df))):

        ch = chunk_text(df['data'][i])

        em = model.encode(ch)

        for it, e in enumerate(em):
            chunked_text.append(ch[it])

            docs.append(f'doc{i}_chunk{it}')
            embeds.append(e)
            metas.append({'label': df['labels'][i]})
            ids.append(f'id{i}_{it}')

    if add_to:
        collection.add(
            documents=docs,
            embeddings=embeds,
            metadatas=metas,
            ids=ids
        )

    dict_df = {'id': ids, 'chunked_text': chunked_text}

    return pd.DataFrame(dict_df)

In [ ]:
ch_text = add_to_collect(df, collection, True)

In [ ]:
ch_text2 = add_to_collect(df, collection2, True)

ch_text3 = add_to_collect(df, collection3, True)


In [ ]:
collection.count()

In [ ]:
ch_text.to_csv('/content/drive/MyDrive/chunked_text.csv')

In [ ]:
ccc = pd.read_csv('/content/drive/MyDrive/chunked_text.csv')
len(ccc)

In [ ]:
questions = ['Which Harry Potter book is the most popular', 'Which US car firms cut their production', 'Who wins double in Glasgow', 'Members of the Flora London Marathon this year', 'How many text messages was sent by US mobile users in 2004']

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def get_answers(collection, ch_text):
    for q in questions:
        q_e = model.encode(q)
        que = collection.query(
            query_embeddings=q_e,
            include=["documents"]
        )
        res = que['ids'][0][:10]
        for id in res:
            print(ch_text[ch_text['id'] == id]['chunked_text'])
            print('-------------------')
        print()
        print()

In [ ]:
get_answers(collection, ch_text)

In [ ]:
get_answers(collection2, ch_text2)

In [ ]:
get_answers(collection3, ch_text3)